<a href="https://colab.research.google.com/github/DJCordhose/llm-ops/blob/main/Present_Lllama_3.2_1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Lingual Medical Assessment Classification

## LLama 3.1 8B
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

### Measurements

#### 4 Bit T4
* Negatives: 35 s
* Positives: 38 s

#### 4 Bit T4 (optimized config)
* Negatives: 24 s
* Positives: 25 s

#### 8 Bit T4
* Negatives: 23 s
* Positives: 23 s

#### Full Res L4
* Negatives: 12.5 s
* Positives: 12.5 s

## Phi 3.5 MoE
* https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/discover-the-new-multi-lingual-high-quality-phi-3-5-slms/ba-p/4225280
* https://huggingface.co/microsoft/Phi-3.5-mini-instruct
* https://huggingface.co/microsoft/Phi-3.5-MoE-instruct

### Measurements

#### 4 Bit A100
* Negatives: 50 s
* Positives: 50 s

## Memory consumption and Quantization

### Quantization 4-Bit / 8-Bit
* 4-Bit (FP4): https://huggingface.co/blog/4bit-transformers-bitsandbytes
  * computation is not done in 4bit, the weights and activations are compressed to that format and the computation is still kept in native dtype
* 8-Bit (LLM.int8()): https://huggingface.co/blog/hf-bitsandbytes-integration#a-gentle-summary-of-llmint8-zero-degradation-matrix-multiplication-for-large-language-models
  * performs the matrix multiplication of the outliers in FP16 and the non-outliers in int8

### Memory consumption for Lllama 3.1 models
- How much memory does Llama 3.1 need for weights and cache (depending on the actually used context length) https://huggingface.co/blog/llama31#inference-memory-requirements
- Detailed, but still comprehensive explanation of how inference in LLMs works: https://developer.nvidia.com/blog/mastering-llm-techniques-inference-optimization/


# Hands-on: Inference on commodity hardware

**Run the assessment classification on the best Llama 3.x model possible**

1. Get a very subjective impression: Do you like the results in terms of e.g.
   1. Quality of language
   1. Halluzination
   1. Accuracy
   1. Speed
   1. Memory consumption
1. If more than one Llama 3.x model actually runs, subjectively compare the results

## Optional
1. Compare to Phi Mini
1. Try a different langauge (German is prepared)


In [1]:
%%time

!pip install --upgrade -q transformers accelerate torch bitsandbytes

CPU times: user 17.5 ms, sys: 5.67 ms, total: 23.2 ms
Wall time: 2.81 s


In [2]:
# Can take forever, and we can do without it

# !pip install --upgrade flash_attn

In [3]:
import transformers

transformers.__version__

'4.46.0'

In [4]:
from google.colab import userdata

In [5]:
# Configure HuggingFace token as a Colab Secret, use key symbol on the left panel
!huggingface-cli login --token {userdata.get('HF_TOKEN')}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
from IPython.display import Markdown

In [8]:
!nvidia-smi

Sun Oct 27 22:14:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   71C    P8              20W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
# kind = 'Lllama_3.1_8B_4bit'
# kind = 'Lllama_3.1_8B_8bit'
# kind = 'Lllama_3.1_8B_16bit'
# kind = 'Lllama_3.2_3B_16bit'
kind = 'Lllama_3.2_1B_16bit'
# kind = 'Phi-3.5-MoE_4bit'
# kind = "Phi-3.5-mini_16bit"

# lang = "de"
lang = "en"

In [10]:
if "Lllama_3.1_8B" in kind:
  model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
elif "Lllama_3.2_3B" in kind:
  model_id = "meta-llama/Llama-3.2-3B-Instruct"
elif "Lllama_3.2_1B" in kind:
  model_id = "meta-llama/Llama-3.2-1B-Instruct"
elif "Phi-3.5-MoE" in kind:
  model_id = "microsoft/Phi-3.5-MoE-instruct"
else:
  model_id = "microsoft/Phi-3.5-mini-instruct"

print(model_id)

meta-llama/Llama-3.2-1B-Instruct


In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [12]:
%%time

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch_dtype = None
quantization_config = None


if "8bit" in kind:
  print("Using 8Bit quantization")
  quantization_config = BitsAndBytesConfig(load_in_8bit=True)
elif "4bit" in kind:
  print("Using 4Bit quantization")
  # quantization_config = BitsAndBytesConfig(load_in_4bit=True)
  quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
  )
else:
  print("Using Full Resolution")
  torch_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch_dtype,
    device_map="cuda",
    trust_remote_code=True
)

Using Full Resolution
CPU times: user 6.22 s, sys: 764 ms, total: 6.98 s
Wall time: 6.19 s


In [13]:
!nvidia-smi

Sun Oct 27 22:14:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   72C    P0              35W /  72W |   2551MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
positive_en = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [15]:
negative_en = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [16]:
positive_de = [
  "Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.",
  "Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.",
  "Alltagsrelevante Gebrauchsvorteile werden festgestellt.",
  "Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.",
  "Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor."
]

In [17]:
negative_de = [
  "Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.",
  "Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.",
  "Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.",
  "In der Gesamtschau der hier vorliegenden Informationen kann nicht erkannt werden, wie die Versorgung des Versicherten mit dem Produkt begründet werden könnte, noch kann die Unbedenklichkeit einer solchen Versorgung bestätigt werden.",
  "Eine ärztliche Begründung, warum im vorliegenden Fall ein nicht im Hilfsmittelverzeichnis gelistetes Produkt zum Einsatz kommen soll, wird nicht übermittelt."
]

In [18]:

if lang == "de":
  negative = negative_de
  positive = positive_de
else:
  negative = negative_en
  positive = positive_en

In [19]:
idx = 0
# assessment = negative[idx]
assessment = positive[idx]

assessment

'With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.'

In [20]:
%%time

if lang == "de":
  messages = [
    {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
    {"role": "user", "content": f'''
    Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

    # Assessment
    {assessment}

    '''}
  ]
else:
  messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 1.52 s, sys: 210 ms, total: 1.73 s
Wall time: 1.77 s


Yes
Justification: A positive assessment indicates that the need for compensation is conceivable, suggesting that the individual may be eligible for benefits under the statutory health insurance system.

In [21]:
def eval_assessment(assessment):
  if lang == "de":
    yes = "Ja"
    no = "Nein"
    messages = [
  {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
  {"role": "user", "content": f'''
  Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

  # Assessment
  {assessment}

  '''}
  ]
  else:
    yes = "Yes"
    no = "No"
    messages = [
      {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
      {"role": "user", "content": f'''
  What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

  # Assessment
  {assessment}

  '''}
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith(yes):
    return "Positive", result
  elif result.startswith(no):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [22]:
%%time

negative_results = []
negative_explanations = []

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  negative_results.append(result)
  negative_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: No specific findings can be derived from the diagnosis currently named as the basis for the regulation.
Positive: Yes
Justification: The assessment indicates that no specific findings can be derived from the diagnosis currently named as the basis for the regulation. This suggests that the diagnosis is not sufficient to trigger the regulation, and therefore, no positive recommendation is given.
-----
Assessment: According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.
Positive: Yes
Justification: The assessment indicates that the insured has been provided with the functional product requested according to its area of application, which suggests that the assessment is positive.
-----
Assessment: A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulat

## Positive

In [23]:
%%time

positive_results = []
positive_explanations = []

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  positive_results.append(result)
  positive_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.
Positive: Yes
Justification: A positive assessment indicates that the need for compensation is conceivable, suggesting that the individual may be eligible for benefits under the statutory health insurance system.
-----
Assessment: The socio-medical prerequisites for the prescribed aid supply have been met.
Positive: Yes
Justification: The assessment indicates that the socio-medical prerequisites for the prescribed aid supply have been met, which typically includes the individual's health status, medical needs, and eligibility for the specific aid.
-----
Assessment: Everyday relevant usage benefits have been determined.
Positive: Yes
Justification: A positive assessment indicates that the assessment has identified benefits that are relevant to everyday life, which is a key aspect of statutory health insurance.
-----
Assessment: Socio-medical indication for the aid is confirmed.


In [24]:
!nvidia-smi

Sun Oct 27 22:15:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   77C    P0              65W /  72W |   2627MiB / 23034MiB |     55%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [25]:
import pandas as pd

df = pd.DataFrame({
    'assesment': negative + positive,
    'y_true': ['Negative'] * len(negative) + ['Positive'] * len(positive),
    'y_hat': negative_results + positive_results,
    'explanation': negative_explanations + positive_explanations
})
df

,assesment,y_true,y_hat,explanation
0,No specific findings can be derived from the d...,Negative,Positive,Yes\nJustification: The assessment indicates t...
1,According to the service extracts from the hea...,Negative,Positive,Yes\nJustification: The assessment indicates t...
2,A medically comprehensible explanation as to w...,Negative,Positive,Yes\nJustification: The assessment concludes t...
3,From an overall view of the information availa...,Negative,Positive,Yes\nJustification: The assessment indicates t...
4,A medical justification for why a product not ...,Negative,Positive,Yes\nJustification: The absence of a medical j...
5,"With the diagnosis named here, the need for co...",Positive,Positive,Yes\nJustification: A positive assessment indi...
6,The socio-medical prerequisites for the prescr...,Positive,Positive,Yes\nJustification: The assessment indicates t...
7,Everyday relevant usage benefits have been det...,Positive,Positive,Yes\nJustification: A positive assessment indi...
8,Socio-medical indication for the aid is confir...,Positive,Positive,Yes\nJustification: A positive assessment indi...
9,Contraindications have been excluded; there ar...,Positive,Positive,Yes\nJustification: The assessment indicates t...


In [26]:
df.to_excel(f'results_{kind}_{lang}.xlsx', index=False)

In [27]:
!ls -l

total 36
-rw-r--r-- 1 root root 6359 Oct 27 21:48 results_Lllama_3.1_8B_16bit_en.xlsx
-rw-r--r-- 1 root root 6465 Oct 27 22:01 results_Lllama_3.1_8B_4bit_en.xlsx
-rw-r--r-- 1 root root 6333 Oct 27 21:53 results_Lllama_3.1_8B_8bit_en.xlsx
-rw-r--r-- 1 root root 6202 Oct 27 22:15 results_Lllama_3.2_1B_16bit_en.xlsx
drwxr-xr-x 1 root root 4096 Oct 24 13:20 sample_data
